In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
csvfile = "./decorated_data.csv"
df = pd.read_csv(csvfile,dtype=object)

In [3]:
df.head()

,ticker,cusip,indsec,indgrp
0,T,00206R102,Public Utilities,Telecommunications Equipment
1,AAL,02376R102,Transportation,Air Freight/Delivery Services
2,AXP,025816109,Finance,Finance: Consumer Services
3,AAPL,037833100,Technology,Computer Manufacturing
4,BK,064058100,Finance,Major Banks


In [4]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [5]:
# create an engine to sqlite db
engine = create_engine("sqlite:///sec13f.sqlite")

In [6]:
# Create a connection to the engine called conn
conn = engine.connect()

In [7]:
# Use `declarative_base` from SQLAlchemy to connect your class to your sqlite database
Base = declarative_base()

In [8]:
class SecuritiesEx(Base):
    __tablename__ = 'securitiesex'
    id = Column(Integer, primary_key=True)
    ticker = Column(Text)
    cusip = Column(Text)
    indsec = Column(Text)
    indgrp = Column(Text)
    
    def __repr__(self):
        return f"indsec={self.indsec}, indgrp={self.indgrp}, cusip={self.cusip} ticker={self.ticker}"
    

In [9]:
# Use `create_all` to create the indsectorindgroup table in the database
Base.metadata.create_all(engine)

In [10]:
# Use MetaData from SQLAlchemy to reflect the tables\n",
metadata = MetaData(bind=engine)
metadata.reflect()

In [12]:
# Save the reference to the `securitiesex` table as a variable called `table`
table = sqlalchemy.Table('securitiesex', metadata, autoload=True)

In [13]:
# Use `table.delete()` to remove any existing data.
conn.execute(table.delete())

In [14]:
data = df.to_dict(orient='records')
print(data)

[{'ticker': 'T', 'cusip': '00206R102', 'indsec': 'Public Utilities', 'indgrp': 'Telecommunications Equipment'}, {'ticker': 'AAL', 'cusip': '02376R102', 'indsec': 'Transportation', 'indgrp': 'Air Freight/Delivery Services'}, {'ticker': 'AXP', 'cusip': '025816109', 'indsec': 'Finance', 'indgrp': 'Finance: Consumer Services'}, {'ticker': 'AAPL', 'cusip': '037833100', 'indsec': 'Technology', 'indgrp': 'Computer Manufacturing'}, {'ticker': 'BK', 'cusip': '064058100', 'indsec': 'Finance', 'indgrp': 'Major Banks'}, {'ticker': 'LBRDA', 'cusip': '16117M305', 'indsec': 'Consumer Services', 'indgrp': 'Television Services'}, {'ticker': nan, 'cusip': '16119P108', 'indsec': 'None', 'indgrp': 'None'}, {'ticker': 'CBI', 'cusip': '167250109', 'indsec': 'Basic Industries', 'indgrp': 'Engineering & Construction'}, {'ticker': 'KO', 'cusip': '191216100', 'indsec': 'Consumer Non-Durables', 'indgrp': 'Beverages (Production/Distribution)'}, {'ticker': 'COP', 'cusip': '20825C104', 'indsec': 'Energy', 'indgrp':

In [15]:

conn.execute(table.insert(), data)

In [16]:
conn.execute("select * from securitiesex").fetchall()

[(1, 'T', '00206R102', 'Public Utilities', 'Telecommunications Equipment'),
 (2, 'AAL', '02376R102', 'Transportation', 'Air Freight/Delivery Services'),
 (3, 'AXP', '025816109', 'Finance', 'Finance: Consumer Services'),
 (4, 'AAPL', '037833100', 'Technology', 'Computer Manufacturing'),
 (5, 'BK', '064058100', 'Finance', 'Major Banks'),
 (6, 'LBRDA', '16117M305', 'Consumer Services', 'Television Services'),
 (7, None, '16119P108', 'None', 'None'),
 (8, 'CBI', '167250109', 'Basic Industries', 'Engineering & Construction'),
 (9, 'KO', '191216100', 'Consumer Non-Durables', 'Beverages (Production/Distribution)'),
 (10, 'COP', '20825C104', 'Energy', 'Integrated oil Companies'),
 (11, 'COST', '22160K105', 'Consumer Services', 'Department/Specialty Retail Stores'),
 (12, 'DVA', '23918K108', 'Health Care', 'Hospital/Nursing Management'),
 (13, 'DE', '244199105', 'Capital Goods', 'Industrial Machinery/Components'),
 (14, 'DAL', '247361702', 'Transportation', 'Air Freight/Delivery Services'),
 (1

In [17]:
conn.close()